# Data wrangling of the Quotebank dataset

This notebook is dedicated to developpe the datawrangling methods that will be used to clean the full Quotebank dataset used in our project. The methods are tested on a sample of the full database.

The sample is made frome 200'000 quotes of each years of interest (2015-2020) which should therefore contain $1.2 \cdot 10^6$ entries.

The script will proceed through the following steps:
  - drop of the duplicates
  - drop of the quotes for which the speaker is not identified (threshold to be defined)
  - drop the quotes in which the probability between one or more speaker his near each other (threshold to be defined)
  - drop of any quotes that is empty
  - keep only the speaker that has the most probability of being the author of the quote
    

In [136]:
#importing the required modules
import seaborn as sns
from IPython.display import display, HTML
import matplotlib.pyplot as plt
import scipy.stats as stats
import pandas as pd
import numpy as np
import seaborn as sns

In [137]:
# Small adjustments to default style of plots, making sure it's readable and colorblind-friendly everywhere
plt.style.use('seaborn-colorblind')
plt.rcParams.update({'font.size' : 12.5,
                     'figure.figsize':(10,7)})

Quick look at the raw data:

In [138]:
%%time
#copy the path of the sample quotes: (to big to put in the git)
#ALEX: 'C:/Users/alexb/Documents/Ecole/EPFL/MasterII/ADA/'
#JULES: ...
#MARIN: ...
#NICO: ...


path_2_data = 'C:/Users/alexb/Documents/Ecole/EPFL/MasterII/ADA/'


#import the dataset sample
raw_data = pd.read_json(path_2_data + 'Sample.json.bz2',compression="bz2",lines=True)

raw_data.describe()

Wall time: 1min 27s


,numOccurrences
count,1.263790e+06
mean,3.767778e+00
std,4.666187e+01
min,1.000000e+00
25%,1.000000e+00
50%,1.000000e+00
75%,2.000000e+00
max,3.300000e+04


In [139]:
raw_data.sample(3)

,quoteID,quotation,speaker,qids,date,numOccurrences,probas,urls,phase
825679,2019-12-14-052319,What I will say is the one thing Trump and Bor...,Jonathan Tasini,[Q6274567],2019-12-14 11:03:39,2,"[[Jonathan Tasini, 0.7711], [None, 0.2089], [B...",[http://thehill.com/homenews/campaign/474492-d...,E
137980,2015-08-12-060985,Just so this doesn't escalate and we end up wi...,Stephen Parry,"[Q3973213, Q53575550, Q7610199, Q7610200]",2015-08-12 01:05:32,4,"[[Stephen Parry, 0.9712], [None, 0.0288]]",[http://geelongadvertiser.com.au/news/national...,E
1116621,2020-03-14-026016,Should there be an incident of an infected att...,Wilson Lim,[Q56777583],2020-03-14 23:02:43,2,"[[Wilson Lim, 0.9647], [None, 0.0353]]",[http://www.smh.com.au/national/queensland/bri...,E


Test to see if the ids are unique within the dataset

In [140]:
%%time
#Keeping the first occurence of the duplicates
size_bf = raw_data.shape[0]
df = raw_data.copy().drop_duplicates(subset = 'quoteID', keep='first')
size_af = df.shape[0]

print('{} dupplicates rows have been removed'.format(size_bf-size_af))
print('Unique rows in the data set:', df.quoteID.is_unique)

0 dupplicates rows have been removed
Unique rows in the data set: True
Wall time: 1.62 s


Sample of the sample to speed up calculation while implementing

In [161]:
df_test = df[0:400000]
df_test.sample(2)

,quoteID,quotation,speaker,qids,date,numOccurrences,probas,urls,phase
266672,2016-05-29-013634,He is very well liked and respected.,Pat Russell,[Q7143980],2016-05-29 17:50:02,1,"[[Pat Russell, 0.7907], [None, 0.2003], [Glen ...",[http://thestarpress.com/story/news/local/2016...,E
167699,2015-03-16-001466,"a strong fiscal and social, pro-life",Phil Pavlov,[Q18763623],2015-03-16 20:02:06,1,"[[Phil Pavlov, 0.8783], [None, 0.1102], [Candi...",[http://freep.com/story/news/local/2015/03/16/...,E


Drop of the quotes for which the speaker:
 - is not clearly identified (threshold to be defined: *threshold_min*)
 - is probably confused with another speaker (threshold to be defined: *threshold_diff*)
 - is not identified (None values)

In [162]:
%%time
#fixing the threshold (=> to low percentage of attribution to be considered in the analysis)
threshold_min = 0.5
threshold_diff = 0.3

#removing the rows that not pass the criterions
df_thres = df_test.copy()
df_thres['p1'] = [i[0][1] for i in df_test['probas']]
df_thres['p2'] = [i[1][1] for i in df_test['probas']]
df_thres['delta_p'] = df_thres['p1'].astype("float")-df_thres['p2'].astype("float")

indexNames = df_thres[
                        (df_thres['p1'].astype("float") < threshold_min)
                      | (df_thres['delta_p'] < threshold_diff)
                      | (df_thres['speaker'] == 'None')
                                                                          ].index

df_thres.drop(indexNames , inplace=False)

#check of the number of deleted rows
print('Min Threshold set at ',threshold_min)
print('Diff Threshold set at ',threshold_diff)
print('----------------------')
print('Due to the above threshold, {} rows have been removed'.format(len(indexNames)))
print(' - {} x None value for speaker'.format(df_test['speaker'].value_counts()['None']))
print(' - {} x threshold probabilities not reached'.format(len(indexNames)-df_test['speaker'].value_counts()['None']))

Min Threshold set at  0.5
Diff Threshold set at  0.3
----------------------
Due to the above threshold, 186643 rows have been removed
 - 135559 x None value for speaker
 - 51084 x threshold probabilities not reached
Wall time: 2.14 s


In [168]:
df_thres.sample()

,quoteID,quotation,speaker,qids,date,numOccurrences,probas,urls,phase,p1,p2,delta_p
129798,2015-10-30-079184,Our liquefied natural gas operations had a rob...,Helge Lund,[Q718292],2015-10-30 11:10:34,1,"[[Helge Lund, 0.8674], [None, 0.1326]]",[http://www.morningstar.co.uk/uk/news/AN_14462...,E,0.8674,0.1326,0.7348
